For this case study, you will perform a classification task on a WiFi dataset. You will use WLAN fingerprints to identify the location of a user. You will identify locations using the building numbers and floor numbers only.

You will also explore the question, "is more data useful for a classification task?"

The dataset you will use can be found on: https://archive.ics.uci.edu/ml/datasets/ujiindoorloc .

**\[Step 1\]** Once you examine the data sets, you will find that there is a training set and a validation set. However, you must also create a test set that has the same number of samples as the validation set. You can select and remove random samples from the training set and use them to create a test set. The test set should not be used in the training process or to optimize the parameters of any algorithm you use. The test set should only be used to report the final performance of a model whenever necessary.

You may need to determine the features and labels of your model. You can also do some engineering on features and labels if necessary.

**\[Step 2\]** But, which algorithm should you use with your model? You can refer to the scikit-learn cheat sheet: https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html, and try three algorithms. Some suggestions are: LinearSVC, Logistic Regression, KNN classifier, SVC, Random Forest (as an example of Ensemble Learning) etc. Perform one experiment using each and observe the performance of each model. Note which is the best performing model using the test set.

**\[Step 3\]** Once the previous step is done, observe if more data is useful for a classification task. For this, randomly select 20% of the training samples, but keep the size of the test set the same. You wil not use the validation set in this step as you will not optimize the model in any way. Note the performance. Then also try with 40%, 60%, 80% and 100% of the training samples. Perform three experiments for each size of the training set. This means, for 20% you will do three experiments, 40% three experiments etc. Find the average of three experiments for each size and plot them using a method of your choice.

**\[Step 4\]** Publish your finding in presentation slides. Like case study 1, three of you will be randomly chosen to present your work in front of the class. The slides should inform the audience about:

* the objective of the case study
* the data (features and labels)
* things you have done (e.g. why you selected a specific classification model)
* challenges you have faced that might be interesting to your classmates
* your findings


**Things to note**:

* **Type of task**: classification
* **Features**: you choose
* **Feature engineering**: You are welcome to do so.
* **Labels**: User locations. Use building and floor IDs, but ignore the SPACEID column.

* In some cases, normalization may result in reduced accuracy.
* You must write enough comments so that anybody with some programming knowledge can understand your code.

Also,
* This is not a group project. But if you think you will benefit from working with a partner, you are welcome to find a partner. In that case, please send an email to the TA and the professor by **September 24, 2024**. Please note that if you choose to work with a partner, it will not impact your score, but you must send the email by September 24. Only one email is enough. Please make sure to CC your partner.

**Grading Criteria**:

* [15 + 15] Data set preparation: Choosing your $X$ (features) and $y$ (label). Feature Engineering.
* [15 + 15 + 15] Three experiments using three algorithms.  
* [15] Observing the effects of more data using five sets of random samples of different sizes from the training set.
* [10] Presentation slides and presentation.

**What to submit**:

Put the Jupyter Notebook file and the .csv file in a folder. Then convert your presentation slides to a PDF file and put it in the same folder. Zip the folder. After zipping, it should have the extension .zip. The name of the .zip file should be firstname_lastname_casestudy_2.zip . Upload the .zip file on Canvas.

In [4]:
# start here
# create as many cells as needed


In [5]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt

In [6]:
train_data = pd.read_csv('trainingData.csv')

In [7]:
validation_data = pd.read_csv('validationData.csv')

In [8]:
# Features and labels
X_train = train_data.iloc[:, :520]  # WAP features
y_train = train_data.loc[:, ['BUILDINGID', 'FLOOR']]  # Building ID and Floor as labels

In [9]:
# Handle non-detected WAPs by setting them to a minimum value (e.g., -103)
X_train[X_train == 100] = -103

In [10]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [11]:
# Split some training data to create a new test set
X_train_final, X_test, y_train_final, y_test = train_test_split(X_train_scaled, y_train, test_size=len(validation_data), random_state=42)

In [12]:
# Define models
models = {
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'SVM': SVC(kernel='linear', random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42)
}

In [13]:
# Train models and evaluate
for name, model in models.items():
    # Building ID
    model.fit(X_train_final, y_train_final['BUILDINGID'])
    y_pred_building = model.predict(X_test)
    accuracy_building = accuracy_score(y_test['BUILDINGID'], y_pred_building)

    # Floor
    model.fit(X_train_final, y_train_final['FLOOR'])
    y_pred_floor = model.predict(X_test)
    accuracy_floor = accuracy_score(y_test['FLOOR'], y_pred_floor)

    print(f'{name} - Building ID Accuracy: {accuracy_building}')
    print(f'{name} - Floor Number Accuracy: {accuracy_floor}')

KNN - Building ID Accuracy: 0.9963996399639964
KNN - Floor Number Accuracy: 0.9810981098109811
SVM - Building ID Accuracy: 0.9963996399639964
SVM - Floor Number Accuracy: 0.9900990099009901
Random Forest - Building ID Accuracy: 0.9963996399639964
Random Forest - Floor Number Accuracy: 0.9945994599459946


In [14]:
# Define model constructors in a dictionary
models = {
    'KNN': lambda: KNeighborsClassifier(n_neighbors=5),
    'SVM': lambda: SVC(kernel='linear', random_state=42),
    'Random Forest': lambda: RandomForestClassifier(n_estimators=100, random_state=42)
}

In [15]:
# Adjusted experiment_with_data_sizes function to use model constructors
def experiment_with_data_sizes(X, y, test_set, target, model_constructors, data_sizes):
    results = {}

    for name, model_constructor in model_constructors.items():
        accuracies = []
        for size in data_sizes:
            temp_acc = []
            for _ in range(3):  # Repeat experiment 3 times
                if size < 1.0:
                    X_sample, _, y_sample, _ = train_test_split(X, y[target], train_size=size, random_state=np.random.randint(0, 10000))
                else:
                    X_sample, y_sample = X, y[target]  # Use the full dataset

                model_instance = model_constructor()
                model_instance.fit(X_sample, y_sample)
                y_pred = model_instance.predict(test_set)
                temp_acc.append(accuracy_score(y_test[target], y_pred))

            accuracies.append(np.mean(temp_acc))
        results[name] = accuracies
    return results

In [16]:
# Perform experiments for BUILDINGID and FLOOR with all models
data_sizes = [0.2, 0.4, 0.6, 0.8, 1.0]
results_building = experiment_with_data_sizes(X_train_scaled, y_train, X_test, 'BUILDINGID', models, data_sizes)
results_floor = experiment_with_data_sizes(X_train_scaled, y_train, X_test, 'FLOOR', models, data_sizes)


In [17]:
plt.figure(figsize=(12, 6))

# Plotting results for BUILDINGID
plt.subplot(1, 2, 1)  # Create subplot for BUILDINGID
for name, accuracies in results_building.items():
    plt.plot(data_sizes, accuracies, marker='o', label=f'{name} Building')
plt.xlabel('Training Data Size')
plt.ylabel('Accuracy')
plt.title('Model Accuracy vs. Training Data Size for BUILDINGID')
plt.legend()

In [18]:
# Plotting results for FLOOR
plt.subplot(1, 2, 2)  # Create subplot for FLOOR
for name, accuracies in results_floor.items():
    plt.plot(data_sizes, accuracies, marker='x', linestyle='--', label=f'{name} Floor')
plt.xlabel('Training Data Size')
plt.ylabel('Accuracy')
plt.title('Model Accuracy vs. Training Data Size for FLOOR')
plt.legend()

plt.tight_layout()  # Adjust layout to prevent overlap
plt.show()